# Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle
import wandb

#regression models
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, HistGradientBoostingRegressor, ExtraTreesRegressor
from xgboost.sklearn import XGBRegressor
from lightgbm import LGBMRegressor

# Config

In [2]:
wandb.login(key='b75e0564aba32dce859c60044418df71ce7389a8')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Import Preprocessed Data

In [3]:
data = pd.read_csv('../input/naalaiya-thiran/Preprocessed/autos_preprocessed.csv', header=0, sep=',', encoding='Latin1')

# Label Encoding

In [4]:
labels = ['gearbox', 'notRepairedDamage', 'model', 'brand', 'fuelType', 'vehicleType']

mapper = {}
for i in labels:
    mapper[i] = LabelEncoder()
    mapper[i].fit(data[i])
    tr = mapper[i].transform(data[i])
    np.save(str('classes'+i+'.npy'), mapper[i].classes_)
    data.loc[:, i+'_labels'] = pd.Series(tr, index=data.index)
    
labeled = data[['price', 'yearOfRegistration','powerPS','kilometer','monthOfRegistration']
                  +[x+"_labels" for x in labels]]

print(labeled.columns)

Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


# Score Evaluation

In [5]:
def find_scores(Y_actual, Y_pred, X_train):
    mae = mean_absolute_error(Y_actual, Y_pred)
    mse = mean_squared_error(Y_actual, Y_pred)
    rmse = np.sqrt(mse)
    rmsle = np.log(rmse)
    r2 = r2_score(Y_actual, Y_pred)
    n, k = X_train.shape
    adj_r2_score = 1 - ((1-r2)*(n-1)/(n-k-1))
    
    wandb.log({"mae": mae, "mse": mse, 'rmse':rmse, 'rmsle':rmsle, 'r2':r2, 'adj_r2':adj_r2_score})

# Predictive Modeling

## Bagging Regressor

In [6]:
def bagging_regressor():
    config_defaults = {
                'n_estimators':100,
                'max_samples':0.4,
                'bootstrap':True,
                'random_state':42
            }
    wandb.init(config=config_defaults)
    config = wandb.config
    
    X = labeled.iloc[:,1:].values
    Y = labeled.iloc[:,0].values.reshape(-1,1)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
    
    model = BaggingRegressor(
      n_estimators=config.n_estimators, 
      bootstrap=config.bootstrap,
      max_samples=config.max_samples,
      random_state = config.random_state)
    
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_test)
    
    find_scores(Y_test, Y_pred, X_train)

In [7]:
bagging_regressor_configs = {
    "name":'BaggingRegressor',
    "method": "grid",
    "metric": {
        "name": "adj_r2",
        "goal": "maximize"
    },
    "parameters": {
        "n_estimators": {
            "values": [100, 200, 300]
        },
        "max_samples": {
            "values": [0.4,0.5, 0.6]
        }
    }
}

#sweep_id = wandb.sweep(sweep=bagging_regressor_configs, project="car_resale_value")
#wandb.agent(sweep_id=sweep_id, function=bagging_regressor)

## RandomForest Regressor

In [8]:
def random_forest_regressor():
    config_defaults = {
                'n_estimators':100,
                'max_samples':0.4,
                'criterion':'squared_error',
                'bootstrap': True,
                'random_state':42
            }
    wandb.init(config=config_defaults)
    config = wandb.config
    
    X = labeled.iloc[:,1:].values
    Y = labeled.iloc[:,0].values.reshape(-1,1)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
    
    model = RandomForestRegressor(
      n_estimators=config.n_estimators,
      criterion = config.criterion,
      bootstrap=config.bootstrap,
      max_samples=config.max_samples,
      random_state = config.random_state)
    
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_test)
    
    find_scores(Y_test, Y_pred, X_train)

In [9]:
random_forest_configs = {
    "name":'RandomForestRegressor',
    "method": "grid",
    "metric": {
        "name": "adj_r2",
        "goal": "maximize"
    },
    "parameters": {
        "n_estimators": {
            "values": [100, 200, 300]
        },
        "max_samples": {
            "values": [0.4,0.5, 0.6]
        }
    }
}

#sweep_id = wandb.sweep(sweep=random_forest_configs, project="car_resale_value")
#wandb.agent(sweep_id=sweep_id, function=random_forest_regressor)

## HistGradient Boosting Regressor

In [10]:
def hist_gradient_boost_regressor():
    config_defaults = {
                'loss':'squared_error',
                'learning_rate': 0.1,
                'max_iter':100,
                'random_state':42
            }
    wandb.init(config=config_defaults)
    config = wandb.config
    
    X = labeled.iloc[:,1:].values
    Y = labeled.iloc[:,0].values.reshape(-1,1)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
    
    model = HistGradientBoostingRegressor(
      loss=config.loss,
      learning_rate = config.learning_rate,
      max_iter=config.max_iter,
      random_state = config.random_state)
    
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_test)
    
    find_scores(Y_test, Y_pred, X_train)

In [11]:
hist_gradient_boost_configs = {
    "name":'HistGradientBoostingRegressor',
    "method": "grid",
    "metric": {
        "name": "adj_r2",
        "goal": "maximize"
    },
    "parameters": {
        "loss": {
            "values": ['squared_error', 'absolute_error']
        },
        "learning_rate": {
            "values": [0.01, 0.03, 0.05, 0.07]
        },
        "max_iter": {
            "values": [100,200,300]
        },
        "random_state": {
            "values": [42]
        }
    }
}

#sweep_id = wandb.sweep(sweep=hist_gradient_boost_configs, project="car_resale_value")
#wandb.agent(sweep_id=sweep_id, function=hist_gradient_boost_regressor)

## Extra Trees Regressor

In [12]:
def extra_tree_regressor():
    config_defaults = {
                'criterion':'squared_error',
                'max_samples':0.4,
                'bootstrap': True,
                'random_state':42
            }
    wandb.init(config=config_defaults)
    config = wandb.config
    
    X = labeled.iloc[:,1:].values
    Y = labeled.iloc[:,0].values.reshape(-1,1)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
    
    model = ExtraTreesRegressor(
      criterion=config.criterion,
      bootstrap = config.bootstrap,
      max_samples=config.max_samples,
      random_state = config.random_state)
    
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_test)
    
    find_scores(Y_test, Y_pred, X_train)

In [13]:
extra_tree_configs = {
    "name":'ExtraTreesRegressor',
    "method": "grid",
    "metric": {
        "name": "adj_r2",
        "goal": "maximize"
    },
    "parameters": {
        "criterion": {
            "values": ['squared_error', 'absolute_error']
        },
        "max_samples": {
            "values": [0.4,0.5, 0.6]
        }
    }
}

#sweep_id = wandb.sweep(sweep=extra_tree_configs, project="car_resale_value")
#wandb.agent(sweep_id=sweep_id, function=extra_tree_regressor)

## XGB Regressor

In [14]:
def XGB_regressor():
    config_defaults = {
                'learning_rate':0.1,
                'n_estimators': 500,
                'booster':'gbtree',
                'eta':0.01,
                'random_state':42
            }
    wandb.init(config=config_defaults)
    config = wandb.config
    
    X = labeled.iloc[:,1:].values
    Y = labeled.iloc[:,0].values.reshape(-1,1)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
    
    model = XGBRegressor(
      learning_rate=config.learning_rate,
      n_estimators = config.n_estimators,
      random_state = config.random_state)
    
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_test)
    
    find_scores(Y_test, Y_pred, X_train)

In [15]:
extra_tree_configs = {
    "name":'XGBRegressor',
    "method": "grid",
    "metric": {
        "name": "adj_r2",
        "goal": "maximize"
    },
    "parameters": {
        "learning_rate": {
            "values": [0.01, 0.03, 0.05, 0.07]
        },
        "n_estimators": {
            "values": [100,200,300]
        },
        "booster": {
            "values": ['gbtree','gblinear']
        },
        "eta": {
            "values": [0.01, 0.03, 0.05, 0.07]
        }
    }
}

sweep_id = wandb.sweep(sweep=extra_tree_configs, project="car_resale_value")
wandb.agent(sweep_id=sweep_id, function=XGB_regressor)

Create sweep with ID: e1cz0udw
Sweep URL: https://wandb.ai/avinash-218/car_resale_value/sweeps/e1cz0udw


wandb: Agent Starting Run: wiw53qhb with config:
wandb: 	booster: gbtree
wandb: 	eta: 0.01
wandb: 	learning_rate: 0.01
wandb: 	n_estimators: 100
wandb: Currently logged in as: avinash-218. Use `wandb login --relogin` to force relogin


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.57395
mae,2765.45081
mse,30367983.19601
r2,0.57397
rmse,5510.71531


wandb: Agent Starting Run: viebru8s with config:
wandb: 	booster: gbtree
wandb: 	eta: 0.01
wandb: 	learning_rate: 0.01
wandb: 	n_estimators: 200


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.77045
mae,1865.54652
mse,16361825.74183
r2,0.77046
rmse,4044.97537


...

wandb: Agent Starting Run: bi8s48j6 with config:
wandb: 	booster: gblinear
wandb: 	eta: 0.07
wandb: 	learning_rate: 0.07
wandb: 	n_estimators: 200


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.86422
mae,1382.56945
mse,9678075.90126
r2,0.86423
rmse,3110.96061


wandb: Agent Starting Run: 5vhj05wr with config:
wandb: 	booster: gblinear
wandb: 	eta: 0.07
wandb: 	learning_rate: 0.07
wandb: 	n_estimators: 300


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.86844
mae,1351.82687
mse,9377218.84143
r2,0.86845
rmse,3062.22449


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## LGBM Regressor

In [16]:
def LGBM_regressor():
    config_defaults = {
                'objective':'root_mean_squared_error',
                'reg_sqrt': True,
                'metric':'rmse',
                'random_state':42
            }
    wandb.init(config=config_defaults)
    config = wandb.config
    
    X = labeled.iloc[:,1:].values
    Y = labeled.iloc[:,0].values.reshape(-1,1)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)
    
    model = LGBMRegressor(
      learning_rate=config.learning_rate,
      n_estimators = config.n_estimators,
      random_state = config.random_state)
    
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_test)
    
    find_scores(Y_test, Y_pred, X_train)

In [17]:
lgbm_configs = {
    "name":'LGBMRegressor',
    "method": "grid",
    "metric": {
        "name": "adj_r2",
        "goal": "maximize"
    },
    "parameters": {
        "learning_rate": {
            "values": [0.01, 0.03, 0.05, 0.07]
        },
        "objective": {
            "values": ['root_mean_squared_error']
        },
        "boosting_type": {
            "values": ['gbdt','dart','goss','rf']
        },
        "reg_sqrt": {
            "values": [True]
        },
        "metric": {
            "values": ['rmse']
        },
        "n_estimators": {
            "values": [100,200,300]
        },
        "random_state": {
            "values": [42]
        }
    }
}

sweep_id = wandb.sweep(sweep=lgbm_configs, project="car_resale_value")
wandb.agent(sweep_id=sweep_id, function=LGBM_regressor)

Create sweep with ID: gbjrsj3v
Sweep URL: https://wandb.ai/avinash-218/car_resale_value/sweeps/gbjrsj3v


wandb: Agent Starting Run: j418qout with config:
wandb: 	boosting_type: gbdt
wandb: 	learning_rate: 0.01
wandb: 	metric: rmse
wandb: 	n_estimators: 100
wandb: 	objective: root_mean_squared_error
wandb: 	random_state: 42
wandb: 	reg_sqrt: True


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.63522
mae,2868.23264
mse,26000513.79616
r2,0.63524
rmse,5099.0699


wandb: Agent Starting Run: mb9v863e with config:
wandb: 	boosting_type: gbdt
wandb: 	learning_rate: 0.01
wandb: 	metric: rmse
wandb: 	n_estimators: 200
wandb: 	objective: root_mean_squared_error
wandb: 	random_state: 42
wandb: 	reg_sqrt: True


...

adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.86505
mae,1400.49904
mse,9618944.77269
r2,0.86506
rmse,3101.44237


wandb: Agent Starting Run: g2376l8g with config:
wandb: 	boosting_type: rf
wandb: 	learning_rate: 0.07
wandb: 	metric: rmse
wandb: 	n_estimators: 300
wandb: 	objective: root_mean_squared_error
wandb: 	random_state: 42
wandb: 	reg_sqrt: True


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


adj_r2,▁
mae,▁
mse,▁
r2,▁
rmse,▁
rmsle,▁
adj_r2,0.86949
mae,1366.21695
mse,9302686.85872
r2,0.86949
rmse,3050.03063


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
